<a href="https://colab.research.google.com/github/sarisppp/PJ63_CE18/blob/master/pj63.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt

from tensorflow import  keras
from tensorflow.keras import layers"""

import os, sys
import os.path
import numpy as np
import pandas as pd
import datetime 

#pd.core.common.is_list_like = pd.api.types.is_list_like

from pandas_datareader import data as pdr
from tqdm import tqdm, trange

import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Test**

In [ ]:
N = 100
X = np.random.rand(N,1)
Y = np.sin(2 * np.pi * X) + 0.4 * np.random.rand(N,1)
plt.plot(X,Y,'.')
plt.show()

In [ ]:
#  1-100-100-1
inputs = keras.Input(shape=(1,))
d1 = layers.Dense(100,activation='relu')
d2 = layers.Dense(100,activation='relu')
d3 = layers.Dense(1)
outputs = d3(d2(d1(inputs)))
model = keras.Model(inputs,outputs)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.1),loss=keras.losses.MeanSquaredError())






In [ ]:
model.fit(X,Y,epochs=500)

In [ ]:
Z = model.predict(X)
plt.plot(X,Z,'.r')
plt.plot(X,Y,'.')
plt.show()

# Load Data

In [ ]:
df = pd.read_excel("/content/drive/My Drive/Set50_2.xlsx",sep='\t')


In [ ]:
df[30:40]
df.info()


# **Load Data from xlsx file**

In [ ]:
used_features = ['Timestamp','SMA(5)','SMA(12)','SMA(26)','MACD','MACD(10)','RSI','Money(5)']
stock = pd.read_excel("/content/drive/My Drive/Set50_2.xlsx", usecols = used_features)
#print(stock.shape)
stock[40:50]
stock.info()
for i in ["SMA(5)"]:
  stock[i] = stock[i].map(lambda x:x.replace("-",''))
  stock[i] = stock[i].astype(float)


In [ ]:
plt.title("SET50")
plt.xlabel('Timestamp')	
plt.ylabel('Closed')
plt.plot(df.OBL, df.OSL , 'ob')
plt.show()	


In [ ]:
"""stock.info()
stock.dtypes
stock['SMA(5)'] = stock['SMA(5)'].astype(float)"""

## **-------------Indicator-------------**




### Simple Moving Average

In [ ]:
def sma(price_close,day):      
    result = price_close.rolling(window=day).mean()
    return result 

### MACD = Moving Average Convergence/Divergence

In [ ]:
def macd( slow, fast):    
    result = slow-fast  
    return result

In [ ]:
df['SMA5'] = sma(df["SET50.Close"],5)
df['SMA12'] = sma(df["SET50.Close"],12)
df['SMA26'] = sma(df["SET50.Close"],26)

#df.drop(columns=['sma55'],inplace=True)
df["EMA12"] = df["EMA(12)"]


df.head(10)






## **features**

In [ ]:
used_features = ['Timestamp','SET50.Open','SET50.Close','SET50.High','SET50.Low','SMA(5)','SMA(12)','SMA(26)','MACD','MACD(10)','RSI','Money','Money(5)']
dtn = pd.read_excel("/content/drive/My Drive/Set50_2.xlsx",sep='\t', usecols = used_features)
print(dtn.shape)
dtn.head(7)

### Check for Null value and preprocessing data

In [ ]:
for feature in used_features:
    if dtn[feature].isnull().values.any():
        print (feature)

In [ ]:
for feature in ['SET50.Open','SET50.Close','SET50.High','SET50.Low','SMA(5)','SMA(12)','SMA(26)','MACD','MACD(10)','RSI','Money','Money(5)']:
    dtn[feature].fillna(dtn[feature].median(),inplace = True)

In [ ]:
dtn.head(30)


## Condition
    1) หากมูลค่าตามราคาตลาดมีค่ามากกว่ามูลค่าตามราคาตลาดเฉลี่ยระยะเวลา 5วัน จะเข้าเงื่อนไขในกฎข้อ 2, 3 และ 4 ต่อไป เพื่อให้ความสนใจในกรณีการแลกเปลี่ยนซื้อขายที่มีขนาดกลางไปจนถึงขนาดใหญ่ และในกรณีที่มูลค่าตามตลาดในขณะนั้น มีค่านอ้ยกว่ามูลค่าตามตลาดเฉลี่ยระยะเวลา 5 วันหรือเกิดการแลกเปลี่ยนซื้อขายขนาดเล็ก ก็จะทำาการถือหลักทรัพย์นั้นไว้และไม่ทำ การขายหรือซ้ือเพิ่มในกรณีที่มีหลักทรัพย์นั้นในการครอบครอง
    2) หากราคาปิดของหลักทรัพย์มีค่ามากกว่าราคาปิดของหลักทรัพย์เฉลี่ยระยะเวลา 5วัน จะทำ การซ้ือหลักทรัพย์น้นัและในกรณีหากราคาปิดของหลักทรัพย์มีค่าน้อยกว่าราคาปิ ดของหลักทรัพย์เฉลี่ยระยะเวลา 5 วัน จะท าการขายหลักทรัพย์น้นั18
    3) หากค่า MACD เป็นบวก ซึ่งมีค่า EMA(12) มากกว่า EMA(26) แสดงว่า ทิศทางแนวโน้มของราคาหลักทรัพย์ และแรงส่งของราคาหลักทรัพย์ กำลังอยู่ในช่วงขาขึ้น จะทำ การขายหลักทรัพย์นั้น และในกรณีค่า MACD เป็นลบซึ่งมีค่า EMA(12)น้อยกว่า EMA(26) แสดงว่า ทิศทางแนวโน้มของราคาหลักทรัพย์และแรงส่งของราคาหลักทรัพย์กำลังอยู่ในช่วงขาลง และใน 2 กรณีที่กล่าวมาข้างต้น โอกาสที่ค่าMACD จะเปลี่ยนจากบวกเป็นลบหรือลบเป็นบวกเกิดข้ึนไดน้อ้ยมากหรืออาจเกิดได้ช้า ซึ่งราคาของหลักทรัพย์อาจจะตกหรือข้ึนไปมากแลว้ เพื่อเพิ่มโอกาสในการทา กา ไร ผูว้ิจยัจึงไดเ้พิ่มกฏยอ่ ยเขา้ไปว่า หากค่า MACD มีค่าเป็ นบวก และค่าMACD มากกว่า MACD เฉลี่ย ระยะเวลา 10 วนั จะทา การซ้ือหลักทรัพย์น้น ใน ักรณีหากค่า MACD มีค่าเป็ นบวกและค่า MACD น้อยกว่า MACD เฉลี่ยระยะเวลา10 วัน จะท าการขายหลักทรัพย์น้นั และในกรณีหากค่า MACD มีค่าเป็ นลบ จะท าการขายหลักทรัพย์น้นั
    4) หากค่า RSI น้อยกว่า 30 แสดงว่าหลักทรัพย์น้นัอาจมีแนวโนม้ ในการพลิกจากช่วงขาลงมาเป็ นช่วงขาข้ึนได้ และในกรณีหากค่า RSI มากกว่า 70 หลักทรัพย์น้นัอาจมีแนวโนม้ ในการพลิกจากช่วงขาข้ึนมาเป็นช่วงขาลงได้ผูว้ิจยัจึงคิดกฏว่า หากจะทาการซ้ือหลักทรัพย์น้นั ไดก้็ต่อเมอื่ ค่า RSI น้อยกว่า 70 และในกรณีจะขายหลักทรัพย์น้นั จะสามารถขายได้เมื่อค่า RSI มากกว่า 30จากเงื่อนไขที่กล่าวมาข้างต้น หากเข้าเงื่อนไขตามกฎข้อ1 คือ 



---

    มูลค่าตามราคาตลาดมีค่ามากกว่ามูลค่าตามราคาตลาดเฉลี่ยระยะเวลา 5 วัน จะสามารถสรุปการเพิ่มค่ากลุ่มแนวโนม้ของหลักทรัพย์ได้ดังนี้
    1) หากเงื่อนไขจากกฎข้อ 2, 3 และ 4 ท้้ง 3 ข้อ ตรงกบั เงื่อนไขการซ้ือ จะเพิ่มค่ากลุ่มแนวโน้มของหลักทรัพย์เป็นการแนะนำให้ซ้ือหลักทรัพย์น้้น ในวัดถัดไป
    2) หากเงื่อนไขจากกฎข้อ 2, 3และ 4 ท้งั 3ขอ้ ตรงกบั เงื่อนไขการขาย จะเพิ่มค่ากลุ่มแนวโน้มของหลักทรัพย์เป็ นการแนะน าให้ขายหลักทรัพย์น้นั ในวนัถดัไป
    3) หากเงื่อนไขจากกฎข้อ 2, 3 และ 4 ท้งั 3 ขอ้ ไม่ตรงเงื่อนไขซ้ือหรือเงื่อนไขขายท้งัหมด จะไม่กระทา การใดๆ หรือทา การถือหลักทรัพย์น้นั ไวใ้นกรณีที่มีหลักทรัพย์น้นัอยใู่ นการครอบครอง19และหากไม่เข้าเงื่อนไขตามกฎข้อ 1 คือ มูลค่าตามราคาตลาดมีค่าน้อยกว่ามูลค่าตามราคาตลาดเฉลี่ยระยะเวลา 5 วัน จะไม่กระท าการใดๆ หรือท าการถือหลักทรัพย์น้นั ไวใ้นกรณีที่มีหลักทรัพย์น้นัอยใู่ นการครอบครอง


In [ ]:
def sel(money,money5):
  if money < money5:
      result = 'Stay'
  else:
      if close > close5:
         result = 'Buy'
      else:
          result = "Sell"
      if macd>=0.0:
          result = "Buy"
      else:
          result = "Sell"
      if rsi >30 and rsi <70:
          result = 'Buy'
      else:
          result = "Sell"
  return result
    

In [ ]:
plt.plot(dtn["SET50.Close"],dtn["SET50.Open"])

plt.title('SET50')
plt.xlabel('X axis')
plt.ylabel('Y axis')

plt.show()



In [ ]:
sel(dtn["Money"][2],dtn["Money(5)"][2])

# Train

In [ ]:
def encode_mysignal(signal, predict="up"):
  if predict=="up":    
    signal[signal < 4] = 0 
    signal[signal == 4] = 1 # สนใจราคาขึ้นมากสุด 4 วันติดกัน

  elif predict=="down":
    signal[signal > 0] = 1     
    signal[signal==0] = 0 # สนใจราคาลงมากสุด 4 วันติดกัน

  return signal

signal = my_signal(dtn['SET50.Close'])

# ปรับตำแหน่งแถวของ ข้อมูล กับ signal ให้สอดคล้องกัน
signal = signal.shift(-2) # shift ขึ้นไปสองแถว
signal = signal.fillna(0) # ยังมี NaN อยู่แถวแรก แก้ให้เป็น 0
signal = signal[:-2] # cut tail that is NaN

signal = encode_mysignal(signal, predict="up")

df_new = dtn[:-2] # ตัดให้เหลือแถวเท่ากับ signal

assert len(df_new) == len(data[stock_name]) - 2
assert len(signal) == len(data[stock_name]) - 2
assert len(df_new) == len(signal)


print(np.shape(df_new))
print(np.shape(signal))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
df_scaled = sc.fit_transform(dtn)

assert np.shape(df_scaled)  == np.shape(dtn)

labels = signal.to_numpy()
assert len(df_scaled) == len(labels)